In [32]:
import os
import litellm
import json
from pprint import pprint

In [7]:
# 1. Define your actual Python function
def get_weather(location: str) -> str:
    # Call weather API, database, etc.
    return f"Weather in {location}: 25C, sunny"

In [8]:
# 2. Define the tool schema for the LLM
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current weather",
            "parameters": {
                "type": "object",   # ← The parameters are a JSON object (dict in Python)
                "properties": {
                    "location": {"type": "string"}
                },
                "required": ["location"]
            }
        }
    }
]

In [ ]:
api_key = os.getenv("LITELLM_API_KEY")

# 3. LLM decides to call the tool
response = litellm.completion(
    model="gpt-4.1",
    messages=[{"role": "user", "content": "What's the weather in Boston?"}],
    tools=tools,
    api_key=api_key
)

In [35]:
# 4. LLM returns a tool call REQUEST (just JSON, not executed!)
tool_call = response.choices[0].message.tool_calls[0]
print(response.choices[0].message.tool_calls)
print(tool_call.function.name)  # "get_weather"
print(tool_call.function.arguments)  # '{"location": "Boston"}'

[ChatCompletionMessageToolCall(function=Function(arguments='{"location":"Boston"}', name='get_weather'), id='call_EPEdzC4fHTEU7RwmvS0rCmk4', type='function')]
get_weather
{"location":"Boston"}


In [36]:
# 5. YOU manually execute your Python function
args = json.loads(tool_call.function.arguments)
result = get_weather(**args)  # THIS is where your code runs

In [39]:
print(response.choices[0].message)

Message(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(function=Function(arguments='{"location":"Boston"}', name='get_weather'), id='call_EPEdzC4fHTEU7RwmvS0rCmk4', type='function')], function_call=None, provider_specific_fields={'refusal': None}, annotations=[])


In [41]:
# 6. Send the result back to the LLM
messages = [
    {"role": "user", "content": "What's the weather in Boston?"},
    response.choices[0].message,  # Assistant's tool call request
    {
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": result  # Your function's output
    }
]

final_response = litellm.completion(model="gpt-4.1", messages=messages, api_key=api_key)
print(final_response.choices[0].message.content)

The current weather in Boston is sunny with a temperature of 25°C. If you need a forecast for the upcoming days or more details, let me know!


**The conversation history grows like this:**
```
User: "What's the weather in Boston?"
  ↓
Assistant: [tool_call: get_weather(location="Boston")]
  ↓
Tool: "72°F, sunny"
  ↓
Assistant: "The weather in Boston is currently 72°F and sunny!"